In [1]:
from rl_games.torch_runner import Runner
import ray
import yaml
import torch
import matplotlib.pyplot as plt
import gym
from IPython import display
import numpy as np
import onnx
import onnxruntime as ort
%matplotlib inline

In [2]:
ray.init(object_store_memory=1024*1024*1000)

2022-03-30 17:07:45,372	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.81',
 'raylet_ip_address': '192.168.0.81',
 'redis_address': '192.168.0.81:6379',
 'object_store_address': '/tmp/ray/session_2022-03-30_17-07-44_586149_30118/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-30_17-07-44_586149_30118/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-03-30_17-07-44_586149_30118',
 'metrics_export_port': 51920,
 'node_id': '5201885f6d66ffd9d80df73689a2074bd3579425587e3ead7f73f872'}

In [3]:
config_name = 'rl_games/configs/ppo_cartpole.yaml'

In [ ]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    config['full_experiment_name'] = 'cartpole_onnx'
runner = Runner()
runner.load(config)
runner.run({
    'train': True,
})

Started to train
Starting horovod with rank: 0, size: 1


(pid=30273) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=30273) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=30269) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=30269) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=30265) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=30265) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=30274) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=30274) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=30270) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=30270) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=30266) WARNING: CPU random generator seem to be failing

Env info:
{'action_space': Discrete(2), 'observation_space': Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32), 'state_space': None, 'use_global_observations': False, 'agents': 1, 'value_size': 1}
cuda:0
build mlp: 4
build mlp: 4
fps step: 6338.5 fps step and policy inference: 894.1 fps total: 747.4
fps step: 6307.9 fps step and policy inference: 4324.4 fps total: 2221.3
fps step: 6105.2 fps step and policy inference: 4230.0 fps total: 2181.0
fps step: 6285.3 fps step and policy inference: 4337.4 fps total: 2199.7
fps step: 6228.3 fps step and policy inference: 4281.0 fps total: 2180.9
fps step: 6333.0 fps step and policy inference: 4336.4 fps total: 2214.4
fps step: 6025.9 fps step and policy inference: 4164.6 fps total: 2161.0
fps step: 6143.5 fps step and policy inference: 4221.1 fps total: 2162.0
fps step: 6258.0 fps step and policy inference: 4301.2 fps total: 2189.3
fps step: 6312.5 fps step and policy inference: 4335.1 fps total: 2190.2
saving next best rewards

fps step: 5959.2 fps step and policy inference: 4219.3 fps total: 2201.8
saving next best rewards:  [109.374]
=> saving checkpoint 'runs/cartpole_vel_info_30-17-07-47/nn/cartpole_vel_info.pth'
fps step: 5534.0 fps step and policy inference: 4003.9 fps total: 2138.7
fps step: 5863.9 fps step and policy inference: 4097.1 fps total: 2160.2
fps step: 6014.1 fps step and policy inference: 4244.1 fps total: 2181.9
saving next best rewards:  [115.84127]
=> saving checkpoint 'runs/cartpole_vel_info_30-17-07-47/nn/cartpole_vel_info.pth'
fps step: 5911.2 fps step and policy inference: 4119.9 fps total: 2144.1
saving next best rewards:  [117.59933]
=> saving checkpoint 'runs/cartpole_vel_info_30-17-07-47/nn/cartpole_vel_info.pth'
fps step: 5852.1 fps step and policy inference: 4104.2 fps total: 2143.3
saving next best rewards:  [119.24149]
=> saving checkpoint 'runs/cartpole_vel_info_30-17-07-47/nn/cartpole_vel_info.pth'
fps step: 6320.6 fps step and policy inference: 4419.2 fps total: 2219.9
sav

In [ ]:
agent = runner.create_player()
agent.restore('runs/cartpole_onnx/nn/cartpole_vel_info.pth')

import rl_games.algos_torch.flatten as flatten
inputs = {
    'obs' : torch.zeros((1,) + agent.obs_shape).to(agent.device),
    'rnn_states' : agent.states
}
with torch.no_grad():
    adapter = flatten.TracingAdapter(agent.model.a2c_network, inputs,allow_non_tensor=True)
    traced = torch.jit.trace(adapter, adapter.flattened_inputs,check_trace=False)
    flattened_outputs = traced(*adapter.flattened_inputs)
    print(flattened_outputs)
    
torch.onnx.export(traced, *adapter.flattened_inputs, "cartpole.onnx", verbose=True, input_names=['obs'], output_names=['logits', 'value'])

onnx_model = onnx.load("cartpole.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [ ]:
ort_model = ort.InferenceSession("cartpole.onnx")

outputs = ort_model.run(
    None,
    {"obs": np.zeros((1, 4)).astype(np.float32)},
)
print(outputs)

In [ ]:
is_done = False
env = agent.env
obs = env.reset()
#prev_screen = env.render(mode='rgb_array')
#plt.imshow(prev_screen)
total_reward = 0
num_steps = 0
while not is_done:
    outputs = ort_model.run(None, {"obs": np.expand_dims(obs, axis=0).astype(np.float32)},)
    action = np.argmax(outputs[0])
    obs, reward, done, info = env.step(action)
    total_reward += reward
    num_steps += 1
    is_done = done
    screen = env.render(mode='rgb_array')
    #plt.imshow(screen)
    #display.display(plt.gcf())    
    #display.clear_output(wait=True)
print(total_reward, num_steps)
#ipythondisplay.clear_output(wait=True)